In [1]:
# Imports
import sys
sys.path.insert(1, '/users/wx21978/projects/pion-phys/pi0-analysis/analysis/')
import os
import operator
import numpy as np
import awkward as ak
import pandas as pd
import copy
import matplotlib.pyplot as plt
from python.analysis import EventSelection, Plots, vector, PairSelection, Master, PFOSelection
from apps import photon_pairs
import time

In [2]:
plt_conf = Plots.PlotConfig()
plt_conf.SHOW_PLOT = True
plt_conf.SAVE_FOLDER = None
# plt_conf.BINS = 30

In [16]:
_a_ = 1
_b_ = 2
x = 4

eval("(x>_a_)|(x<_b_)")

True

In [13]:
ord('a')

97

In [24]:
chr(97)

'a'

In [15]:
eval("np." + repr(np.array([1,2])))

array([1, 2])

In [1]:
def dumb_sum(num):
    def next_val(index):
        if index >= num:
            return index
        else:
            return index + next_val(index+1)
    return next_val(0)

dumb_sum(4)

10

In [116]:
def insert_values_to_func_str(func_str, values):
    for i in range(len(values)):
        func_str = func_str.replace(f"_{i}_", f"{values[i]}")
    return func_str

def cuts_to_func(values, *operations, func_str=None):
    ops = {
        "==": operator.eq,
        "!=": operator.ne,
        "<": operator.lt,
        "<=": operator.le,
        ">": operator.gt,
        ">=": operator.ge}

    if func_str is None:
        if len(operations) != 1:
            raise ValueError("operations must be sepcified if func_str is not specified")
        else:
            operations = operations[0]
        def cut_func(property_to_cut):
            def next_cut(index):
                curr_cut = ops[operations[index]](property_to_cut, values[index])
                if index <= 0:
                    return curr_cut
                else:
                    return np.logical_and(curr_cut, next_cut(index-1))
            return next_cut(len(values)-1)
        return cut_func
    else:
        formatted_func = insert_values_to_func_str(func_str, values)
        return lambda x: eval(formatted_func)

def cuts_to_str(values, *operations, func_str=None, name_format=False):
    str_ini = ", " if name_format else ""
    if func_str is None:
        if len(operations) != 1:
            raise ValueError("operations must be sepcified if func_str is not specified")
        else:
            operations = operations[0]
        
        if len(values) == 1:
            str_ini = "" if name_format else "x"
            return str_ini + f" {operations[0]} {values[0]}"
        elif len(values) == 2:
            if (">" in operations[0]) and ("<" in operations[1]):
                return str_ini + f"{values[0]} {operations[0].replace('>','<')} x {operations[1]} {values[1]}"
            elif (">" in operations[1]) and ("<" in operations[0]):
                return str_ini + f"{values[1]} {operations[1].replace('>','<')} x {operations[0]} {values[0]}"
        return str_ini + " and ".join([f"(x {op} {val})" for val, op in zip(values, operations)])

    else:
        return str_ini + insert_values_to_func_str(func_str, values)

In [48]:
"x==_a_ | x>_b_".replace(f"_{chr(ord('a') + 0)}_", f"values[{0}]")

'x==values[0] | x>_b_'

In [119]:
print(cuts_to_func([2,5], func_str="(x==_0_) or (x>_1_)")(5))
print(cuts_to_str([2,5], func_str="(x==_0_) or (x>_1_)"))
print(cuts_to_str([2,5], func_str="(x==_0_) or (x>_1_)", name_format=True))

False
(x==2) or (x>5)
, (x==2) or (x>5)


In [117]:
print(cuts_to_func([2,5], [">=", "<"])(5))
print(cuts_to_str([2,5], [">=", "<"]))
print(cuts_to_str([2,5], [">=", "<"], name_format=True))

False
2 <= x < 5
, 2 <= x < 5


In [118]:
print(cuts_to_func([5,10], ["==", ">"])(1))
print(cuts_to_str([5,10], ["==", ">"]))
print(cuts_to_str([5,10], ["==", ">"], name_format=True))

False
(x == 5) and (x > 10)
, (x == 5) and (x > 10)


In [115]:
print(cuts_to_func([10], ["<="])(1))
print(cuts_to_str([10], ["<="]))
print(cuts_to_str([10], ["<="], name_format=True))

True
x <= 10
 <= 10


In [ ]:
def cut_to_func(cut):
    if isinstance(cut, tuple):
        if len(cut) == 1:
            return lambda count: count >= cut[0]
        elif len(cut) == 2:
            return lambda count: np.logical_and(
                count >= min(cut), count <= max(cut))
        else:
            raise ValueError(f"Cut tuple {cut} must contain 1 or 2 values.")
    if isinstance(cut, list):
        if len(cut) == 1:
            return lambda count: count > cut[0]
        elif len(cut) == 2:
            return lambda count: np.logical_and(
                count > min(cut), count < max(cut))
        else:
            raise ValueError(f"Cut list {cut} must contain 1 or 2 values.")
    elif cut is None:
        return lambda count: True
    else:
        return lambda count: count == cut
    
def cut_to_str(cut):
    if isinstance(cut, tuple):
        if len(cut) == 1:
            return f" >= {cut[0]}"
        elif len(cut) == 2:
            return f", {min(cut)} <= X <= {max(cut)}"
        else:
            raise ValueError(f"Cut tuple {cut} must contain 1 or 2 values.")
    if isinstance(cut, list):
        if len(cut) == 1:
            return f" > {cut[0]}"
        elif len(cut) == 2:
            return f", {min(cut)} < X < {max(cut)}"
        else:
            raise ValueError(f"Cut list {cut} must contain 1 or 2 values.")
    elif cut is None:
        return ", no cut"
    else:
        return f" == {cut}"

In [34]:
class MaskIter():
    def __init__(self, start_sigs, masks, init_step=0):
        self.sigs = start_sigs
        self.masks = masks
        self.index = init_step
        self.iter_max = len(masks)
        self.last_sig = self.sigs[0]
        self.simultaneous_masks = []
        if self.iter_max != len(sigs):
            raise ValueError(f"masks and sigs must have the same length: {self.iter_max} and {len(sigs)}")
        if self.index >= self.iter_max:
            raise IndexError(f"Start index {self.index} is out of bounds for masks length of {self.iter_max}")
        return
    
    def check_simul_masks(self, sig1, sig2):
        return (sig1[0] == sig2[0]) and ((sig1[1] == sig2[-1]) or (sig1[1] == -1) or (sig2[1] == -1))

    def check_current_simultaneous(self):
        return self.check_simul_masks(self.last_sig,
                                      self.sigs[self.index])

    def check_next_simultaneous(self):
        if self.index >= self.iter_max-1:
            return False
        return self.check_simul_masks(self.sigs[self.index],
                                      self.sigs[self.index+1])

    def __next__(self):
        if self.index >= self.iter_max:
            raise StopIteration
        simultaneous = self.check_current_simultaneous()
        if self.check_next_simultaneous():
            gen_next_data = lambda m, d: d
        else:
            gen_next_data = lambda m, d: d[m]
        if simultaneous:
            self.simultaneous_masks.append(self.masks[self.index])
            comb_masks = self.simultaneous_masks[0]
            for m in self.simultaneous_masks[1:]:
                comb_masks = np.logical_and(comb_masks, m)
            def apply_mask_func(data):
                return comb_masks, gen_next_data(data)
        else:
            self.simultaneous_masks = []
            self.last_sig = self.sigs[self.index]
            mask = self.masks[self.index]
            def apply_mask_func(data):
                return mask, gen_next_data(data)
        self.index += 1
        return apply_mask_func


class CutHandler():
    def __init__(self):
        self._curr_masks = []
        self._conseq_masks = []
        self._signatures = []
        # self._signature = () # OLD
        self._start_sig = ()
        self._end_sig = ()
        # self._consequtive = () # OLD
        self.concat_index = 0
        self._concat_indicies = [0]
        self._pfos_init = False
        self._pre_pfo_init_masks = []
        self._init_pfo_count = np.inf
        self._last_pfo_counts = None
        self._pfo_table_cols = [
            "Name", "Remaining events", "Percentage of total events remaining",
            "Relative percentage events", "Remaining PFOs",
            "Percentage of total PFOs remaining", "Relative percentage of PFOs",
            "Average PFOs per event"]
        self._event_table_cols = [
            "Name", "Remaining events", "Percentage of total events remaining",
            "Relative percentage events"]
        self._event_inds_indicies = []

        self._table_data = {}
        for col in self._pfo_table_cols:
            self._table_data.update({col:[]})
    
    def __iter__(self):
        return MaskIter(self._signatures, self._masks, init_step=self.curr_concat_index)

    def copy(self):
        return copy.deepcopy(self)

    def _get_mask_signature(self, mask, end=False, update=False):
        flat_array = isinstance(ak.count(mask, axis=0), int)
        if update or (not end):
            pfo_level = -1 if flat_array else ak.count(mask)
            start_sig = (ak.num(mask, axis=0), pfo_level)
            if not update:
                return start_sig
        if flat_array:
            end_sig = (ak.sum(mask), -1)
        else:
            end_sig = (ak.num(mask, axis=0), ak.sum(mask))
        if not update:
            return end_sig
        # if update is True
        self._start_sig = start_sig
        self._end_sig = end_sig
        return
    
    def _get_last_pfo_sig(self):
        if not self._pfos_init:
            return -1, -1
        else:
            last_full = self._table_data["Percentage of total PFOs remaining"]/100
            last_relative = self._table_data["Relative percentage of PFOs"]/100
            last_start = round(last_full * self._init_pfo_count)
            last_end = round(last_start/last_relative)
            return last_start, last_end

    def _check_signature(self, signature):
        # simul_mask = (self._start_sig[0] == signature[0]) and ((start_pfo == signature[-1]) or (start_pfo == -1) or (signature[-1] == -1))
        # good_mask = simul_mask or (self._end_sig[0] == signature[0] and ((end_pfo == signature[-1]) or (end_pfo == -1) or (signature[-1] == -1)))
        good_mask = False
        if signature[-1] != -1:
            if self._start_sig[1] == -1:
                start_pfo, end_pfo = self._get_last_pfo_sig()
            else:
                start_pfo = self._start_sig[1]
                end_pfo = self._end_sig[1]
        else:
            start_pfo = -1
            end_pfo = -1
        if (self._start_sig[0] == signature[0]):
            simul_mask = True
            good_mask = (start_pfo == signature[-1]) or (start_pfo == -1)
        elif (self._end_sig[0] == signature[0]):
            simul_mask = False
            good_mask = (end_pfo == signature[-1]) or (end_pfo == -1)
        if not good_mask:
            raise ValueError(f"Mask signature ({signature[0]} events, {signature[1]} PFOs) does not match"
                             + f" the required signature of ({self._start_sig[0]} events, "
                             + f"{self._start_sig[1]} PFOs) for a simultaenous mask, or ({self._end_sig[0]}"
                             + f" events, {self._end_sig[1]} PFOs) for a consequtive mask.")
        return simul_mask

    # def _validate_signature(self, signature):
    #     if not self._check_signature(signature):
    #         if self._consequtive:
    #             conseq_text = "consequtive masks (each mask has input signature of the output of the previous mask)"
    #         else:
    #             conseq_text = "initially matched masks (each mask must have the same input signature)"
    #         raise ValueError(f"Mask signature ({signature[0]} events, {signature[1]} PFOs) does not match"
    #                             + f" the required signature for {conseq_text}.\nRequired signature is "
    #                             + f"({self._signature[0]} events, {self._signature[1]} PFOs).")

    def _add_initial_data(self, mask):
        self._signatures += [self._get_mask_signature(mask)]
        self._get_mask_signature(mask, update=True)
        table_data = [
            "Initial data",
            ak.num(mask, axis=0),
            100.,
            100.]
        if self._start_sig[1] == -1:
            table_data += [
                "Not supplied","-","-","-"]
        else:
            self._pfos_init = True
            self._init_pfo_count = ak.count(mask)
            self._last_pfo_counts = ak.count(mask, axis=1)
            table_data += [
                ak.count(mask),
                100.,
                100.,
                ak.count(mask)/ak.num(mask, axis=0)]
        self._append_table_data(table_data)
        return

    def _get_remaining_events_data(self):
        this_sig = self._signatures[-1]
        if this_sig[1] == -1:
            curr_remaining = ak.sum(self.get_curr_mask())
        else:
            curr_remaining = ak.num(self.get_curr_mask(), axis=0)
        init_remaining = self._table_data["Remaining events"][0]
        last_remaining = self._table_data["Remaining events"][-1]
        return [curr_remaining,
                100 * curr_remaining/init_remaining,
                100 * curr_remaining/last_remaining]
    
    def _get_remaining_pfos_data(self):
        this_sig = self._signatures[-1]
        if not self._pfos_init:
            if this_sig[1] == -1:
                self._pre_pfo_init_masks += [self.get_curr_mask()]
                return ["-", "-", "-", "-"]
            else:
                self._pfos_init = True
                this_mask = self.get_curr_mask()
                self._init_pfo_count = ak.count(this_mask)
                last_remaining = ak.sum(self._init_pfo_count)
        else:
            last_remaining = self._table_data["Remaining PFOs"][-1]
        if this_sig[1] == -1:
            this_mask = self.get_curr_mask()
            curr_remaining = ak.sum(self._last_pfo_counts[this_mask])
            curr_events_remaining = ak.sum(this_mask)
            return [
                curr_remaining,
                0.,
                100.,
                curr_remaining/curr_events_remaining]
        else:
            this_mask = self.get_curr_mask()
            self._last_pfo_counts = ak.count(this_mask, axis=1)
            curr_remaining = ak.sum(this_mask)
            curr_events_remaining = ak.num(this_mask, axis=0)
        return [curr_remaining,
                100 * curr_remaining/self._init_pfo_count,
                100 * curr_remaining/last_remaining,
                curr_remaining/curr_events_remaining]

    def _append_table_data(self, data):
        for i in range(len(self._pfo_table_cols)):
            self._table_data[self._pfo_table_cols[i]].append(data[i])
        return
    
    def _update_table_data(self, name):
        table_data = [name]
        table_data += self._get_remaining_events_data()
        table_data += self._get_remaining_pfos_data()
        self._append_table_data(table_data)
        self._concat_indicies[-1] += 1
        return

    def concatenate(self, other, return_copy=False):
        if not isinstance(other, CutHandler):
            raise NotImplementedError("CutHandler object can only be added to another CutHandler object")
        # Make sure the new set is consequtive
        if self._check_signature(other._signatures[0]) and (other._signatures[0][1] != -1):
            raise ValueError(f"Combining CutHandler objects requires consequtively matching signatures: "
                             + f"{self._end_sig} and {other._signatures[0]} do not match.")
        if return_copy:
            result = self.copy()
        else:
            result = self
        result._empty_curr_mask_queue()
        result._conseq_masks += other._conseq_masks
        result._curr_masks = other._curr_masks
        result._signatures += other._signatures[1:]
        result._start_sig = other._start_sig
        result.concat_index += 1
        result._concat_indicies += [i + len(result._concat_indicies) for i in other._concat_indicies]
        renormed_data = result._change_table_data_init_count(
            other,
            result._table_data["Remaining events"][0],
            result._init_pfo_count,
            last_pfos_count=result._last_pfo_counts)
        result._pfos_init |= other._pfos_init
        result._last_pfo_counts = other._last_pfo_counts
        for c in result._pfo_table_cols:
            result._table_data[c] += renormed_data[c][1:]
        if return_copy:
            return result
        else:
            return

    def __add__(self, other):
        return self.concatenate(other, return_copy=True)

    def _change_table_data_init_count(
            self, cut_obj,
            new_init_evts, new_init_pfos,
            last_events=None, last_pfos_count=None):
        new_data = cut_obj._table_data.copy()
        evt_counts = new_data["Remaining events"]
        new_evts_full = [100 * c/new_init_evts for c in evt_counts]
        new_evts_rel = new_data["Relative percentage events"]
        if last_events is not None:
            new_evts_rel[0] = 100 * evt_counts[0]/last_events
        pfo_counts = new_data["Remaining PFOs"]
        recreate_pfo_data = False
        if new_init_pfos != "-":
            if "-" not in pfo_counts:
                new_pfos_full = [100 * c/new_init_pfos for c in pfo_counts]
            else:
                # Need to add thing to catch case of second item being events based
                if last_pfos_count is not None:
                    recreate_pfo_data = True
                    counts_to_use = [np.sum(last_pfos_count[mask]) for mask in cut_obj._pre_pfo_init_masks]
                    counts_to_use += pfo_counts[1+len(counts_to_use):]
                    counts_to_use = [np.sum(last_pfos_count)] + counts_to_use
                    pfo_counts = counts_to_use
                    new_pfos_full = [100 * c/new_init_pfos for c in pfo_counts]
                else:
                    new_pfos_full = new_data["Percentage of total PFOs remaining"]
        else:
            new_pfos_full = new_data["Remaining PFOs"]
        if recreate_pfo_data:
            new_pfos_rel = [100.] + [100 * this/last for this, last in zip(counts_to_use[1:], counts_to_use[:-1])]
        else:
            new_pfos_rel = new_data["Relative percentage of PFOs"]
        if last_pfos_count is not None:
            new_pfos_rel[0] = 100 * pfo_counts/np.sum(last_pfos_count)
        if recreate_pfo_data:
            new_ave_pfos = [pfos/evts for pfos, evts in zip(evt_counts, pfo_counts)]
        else:
            new_ave_pfos = new_data["Average PFOs per event"]
        return {
            "Name":new_data["Name"],
            "Remaining events":evt_counts,
            "Percentage of total events remaining":new_evts_full,
            "Relative percentage events":new_evts_rel,
            "Remaining PFOs":pfo_counts,
            "Percentage of total PFOs remaining":new_pfos_full,
            "Relative percentage of PFOs":new_pfos_rel,
            "Average PFOs per event":new_ave_pfos}

    def _empty_curr_mask_queue(self):
        result = self._curr_masks[0]
        for m in self._curr_masks[1:]:
            result = result * m
        self._conseq_masks += [result]
        self._curr_masks = []
        return

    def add_mask(self, mask, name):
        # TODO need to decide if the mask needs to be stored separately:
        # This is needed if PFO is not set, and we are in an events-like (and simultaneous like, but probably ignore that for ease) signature
        this_sig = self._get_mask_signature(mask)
        if len(self._curr_masks) == 0 and len(self._conseq_masks) == 0:
            self._add_initial_data(mask)
        elif not self._check_signature(this_sig):
            self._empty_curr_mask_queue()
        self._curr_masks += [mask]
        # else:
            # If signature matches previous init signature, then add to self._curr_masks
            # Else if it matches the end signature, put the current mask into self._conseq_masks
            #   Reset current masks back to []
            # curr_sig = self._get_mask_signature(mask)
            # if len(self._curr_masks) == 1:
            #     # self._consequtive stores the initial state of
            #     # the first mask until set as a boolean here
            #     consequtive = curr_sig == self._consequtive
            #     if consequtive:
            #         self._signature = self._consequtive
            #     self._consequtive = consequtive
            # self._validate_signature(curr_sig)
            # if self._consequtive:
            #     self._signature = self._get_mask_signature(mask, end=True)
        self._signatures += [self._get_mask_signature(mask, end=True)]
        self._get_mask_signature(mask, update=True)
        self._update_table_data(name)
        return
        
    def get_curr_mask(self):
        if len(self._curr_masks) == 0:
            if len(self._conseq_masks) == 0:
                raise IndexError("No masks added")
            return self._conseq_masks[-1]
        result = self._curr_masks[0]
        for m in self._curr_masks[1:]:
            result = result * m
        return result
        # if len(self._curr_masks) == 0:
        #     raise IndexError("No masks added")
        # if self._consequtive:
        #     # N.B. also entered if only 1 mask is added
        #     # TODO this doesn't work! mask signature needs to be the initial signature still
        #     res = self._curr_masks[0]
        #     for mask in self._curr_masks[1:]:
        #         res = res[mask]
        # else:
        #     event_masks = []
        #     pfo_masks = []
        #     if not self._pfos_init:
        #         res = self._combine_masks(self._curr_masks)
        #     else:
        #         for i, sig in enumerate(self._signatures):
        #             if sig[1] == -1:
        #                 event_masks += [self._curr_masks[i]]
        #             else:
        #                 pfo_masks += [self._curr_masks[i]]
        #         event_mask = self._combine_masks(event_masks)
        #         res = self._combine_masks(pfo_masks)
        #         if event_mask is not None:
        #             res = event_mask * res
        # return res

    def apply_masks(self, data, return_table=False, application_index=0):
        result = data
        for m in self._conseq_masks[application_index:]:
            result = result[m]
        result = result[self.get_curr_mask()]
        if return_table:
            return result, self.get_table()
        else:
            return result

    def get_table(self, initial_concat_index=0, initial_name="Initial data", events_only=False):
        data = self._table_data
        if initial_concat_index > self.concat_index:
            raise IndexError(
                f"Concatenation index {initial_concat_index} out of range, "
                + f"only {self.concat_index} concatenation(s) present.")
        if initial_concat_index > 0:
            start_event_index = self._concat_indicies[initial_concat_index]
            data = self._change_table_data_init_count(
                self,
                data["Remaining events"][start_event_index],
                data["Remaining PFOs"][start_event_index])
            data["Name"][0] = initial_name
        if events_only:
            return pd.DataFrame({key:data[key] for key in self._event_table_cols})
        else:
            return pd.DataFrame(data)



In [4]:
size=1000
test_data = np.random.randint(0, 10, size=size)
curr_i = 0
data = []
while curr_i < size:
    this_size = np.random.randint(0,6)
    data += [test_data[curr_i:min(curr_i + this_size, size)]]
    curr_i += this_size
data = ak.Array(data)
print(data)
mask_1 = data > 2
print(mask_1)
mask_2 = data < 7
print(mask_2)
mask_3 = data >=4
print(mask_3)

[[1, 8, 9, 7, 9], [9, 6, 1], [3, 9], [4, 7, ... [2, 9, 9], [2], [4, 5, 1], [9, 4]]
[[False, True, True, True, True], [True, True, ... [True, True, False], [True, True]]
[[True, False, False, False, False], [False, ... [True, True, True], [False, True]]
[[False, True, True, True, True], [True, True, ... [True, True, False], [True, True]]


In [35]:
cutter_num = CutHandler()
cutter_num.get_curr_mask()

IndexError: No masks added

In [36]:
cutter_num.add_mask(mask_1, ">2")
print(cutter_num.get_curr_mask())
print("1 done")
cutter_num.add_mask(mask_2, "<7")
print(cutter_num.get_curr_mask())
print("2 done")
cutter_num.add_mask(mask_3, ">=4")
print(cutter_num.get_curr_mask())
print("3 done")
print(data[cutter_num.get_curr_mask()])
print(cutter_num._get_mask_signature(data))
print(cutter_num._get_mask_signature(data[cutter_num.get_curr_mask()]))

[[False, True, True, True, True], [True, True, ... [True, True, False], [True, True]]
1 done
[[False, False, False, False, False], [False, ... [True, True, False], [False, True]]
2 done
[[False, False, False, False, False], [False, ... [True, True, False], [False, True]]
3 done
[[], [6], [], [4], [6], [6], [], [], [5, ... [], [5, 6], [], [], [], [], [4, 5], [4]]
(429, 1000)
(429, 301)


In [37]:
print(cutter_num.apply_masks(data))
print(data[cutter_num.get_curr_mask()])
print(data)
cutter_num.get_table()

[[], [6], [], [4], [6], [6], [], [], [5, ... [], [5, 6], [], [], [], [], [4, 5], [4]]
[[], [6], [], [4], [6], [6], [], [], [5, ... [], [5, 6], [], [], [], [], [4, 5], [4]]
[[1, 8, 9, 7, 9], [9, 6, 1], [3, 9], [4, 7, ... [2, 9, 9], [2], [4, 5, 1], [9, 4]]


,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [8]:
cutter_num.get_table(events_only=True)

,Name,Remaining events,Percentage of total events remaining,Relative percentage events
0,Initial data,429,100.0,100.0
1,>2,429,100.0,100.0
2,<7,429,100.0,100.0
3,>=4,429,100.0,100.0


In [9]:
cutter_num.get_table(initial_concat_index=1)

IndexError: Concatenation index 1 out of range, only 0 concatenation(s) present.

In [18]:
dir(cutter_num)
['add_mask',
 'apply_masks',
 'concat_index',
 'concatenate',
 'copy',
 'get_curr_mask',
 'get_table']

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_initial_data',
 '_append_table_data',
 '_change_table_data_init_count',
 '_check_signature',
 '_concat_indicies',
 '_conseq_masks',
 '_curr_masks',
 '_empty_curr_mask_queue',
 '_end_sig',
 '_event_inds_indicies',
 '_event_table_cols',
 '_get_last_pfo_sig',
 '_get_mask_signature',
 '_get_remaining_events_data',
 '_get_remaining_pfos_data',
 '_init_pfo_count',
 '_last_pfo_counts',
 '_pfo_table_cols',
 '_pfos_init',
 '_signatures',
 '_start_sig',
 '_table_data',
 '_update_table_data',
 'add_mask',
 'apply_masks',
 'concat_index',
 'concatenate',
 'copy',
 'get_curr_mask',
 'get_table']

In [38]:
evts_cutter = CutHandler()
evt_1 = ak.num(data, axis=1) > 0
print(evt_1)
evt_2 = ak.num(data, axis=1) <= 3
print(evt_2)
evts_cutter.add_mask(evt_1, "No empty")
print(evts_cutter.get_curr_mask())
print("1 done")
evts_cutter.add_mask(evt_2, " At most 3 in event")
print(evts_cutter.get_curr_mask())
print("2 done")
print(data[evts_cutter.get_curr_mask()])
print(evts_cutter._get_mask_signature(data))
print(evts_cutter._get_mask_signature(evts_cutter.apply_masks(data)))
print(evts_cutter.apply_masks(data))
print(data[evts_cutter.get_curr_mask()])
print(data)
print(evts_cutter.get_table())
evts_cutter.get_table(events_only=True)

[True, True, True, True, True, True, False, ... True, True, True, True, True, True]
[False, True, True, True, True, False, True, ... True, True, True, True, True, True]
[True, True, True, True, True, True, False, ... True, True, True, True, True, True]
1 done
[False, True, True, True, True, False, False, ... True, True, True, True, True, True]
2 done
[[9, 6, 1], [3, 9], [4, 7, 7], [6], [5], ... 8], [2, 9, 9], [2], [4, 5, 1], [9, 4]]
(429, 1000)
(228, 450)
[[9, 6, 1], [3, 9], [4, 7, 7], [6], [5], ... 8], [2, 9, 9], [2], [4, 5, 1], [9, 4]]
[[9, 6, 1], [3, 9], [4, 7, 7], [6], [5], ... 8], [2, 9, 9], [2], [4, 5, 1], [9, 4]]
[[1, 8, 9, 7, 9], [9, 6, 1], [3, 9], [4, 7, ... [2, 9, 9], [2], [4, 5, 1], [9, 4]]
                  Name  Remaining events  \
0         Initial data               429   
1             No empty               351   
2   At most 3 in event               228   

   Percentage of total events remaining  Relative percentage events  \
0                            100.000000  

,Name,Remaining events,Percentage of total events remaining,Relative percentage events
0,Initial data,429,100.000000,100.000000
1,No empty,351,81.818182,81.818182
2,At most 3 in event,228,53.146853,64.957265


In [39]:
evts_cutter.get_table(events_only=True)

,Name,Remaining events,Percentage of total events remaining,Relative percentage events
0,Initial data,429,100.000000,100.000000
1,No empty,351,81.818182,81.818182
2,At most 3 in event,228,53.146853,64.957265


In [40]:
cutter_num.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [46]:
comb_cuts = cutter_num + evts_cutter

['Initial data', '>2', '<7', '>=4', 'No empty', ' At most 3 in event']
[429, 429, 429, 429, 351, 228]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 53.14685314685315]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 64.95726495726495]
[1000, 685, 394, 301, 1000, 450]
[100.0, 68.5, 39.4, 30.1, 100.0, 45.0]
[100.0, 68.5, 57.518248175182485, 76.39593908629442, 100.0, 45.0]
[2.331002331002331, 1.5967365967365967, 0.9184149184149184, 0.7016317016317016, 0.351, 0.5066666666666667]


In [41]:
print(type(cutter_num.copy()))
print(isinstance(cutter_num.copy(), CutHandler))
print(type(evts_cutter.copy()))
print(isinstance(evts_cutter.copy(), CutHandler))

<class '__main__.CutHandler'>
True
<class '__main__.CutHandler'>
True


In [42]:
x = cutter_num.copy()
print(isinstance(x, CutHandler))
x.concatenate(evts_cutter)

True
['Initial data', '>2', '<7', '>=4', 'No empty', ' At most 3 in event']
[429, 429, 429, 429, 351, 228]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 53.14685314685315]
[100.0, 100.0, 100.0, 100.0, 81.81818181818181, 64.95726495726495]
[1000, 685, 394, 301, 1000, 450]
[100.0, 68.5, 39.4, 30.1, 100.0, 45.0]
[100.0, 68.5, 57.518248175182485, 76.39593908629442, 100.0, 45.0]
[2.331002331002331, 1.5967365967365967, 0.9184149184149184, 0.7016317016317016, 0.351, 0.5066666666666667]


In [44]:
x.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.000000,100.000000,1000,100.0,100.000000,2.331002
1,>2,429,100.000000,100.000000,685,68.5,68.500000,1.596737
2,<7,429,100.000000,100.000000,394,39.4,57.518248,0.918415
3,>=4,429,100.000000,100.000000,301,30.1,76.395939,0.701632
4,No empty,351,81.818182,81.818182,1000,100.0,100.000000,0.351000
5,At most 3 in event,228,53.146853,64.957265,450,45.0,45.000000,0.506667


In [43]:
cutter_num.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [15]:
evts_cutter.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.000000,100.000000,Not supplied,-,-,-
1,No empty,351,81.818182,81.818182,-,-,-,-
2,At most 3 in event,228,53.146853,64.957265,-,-,-,-


In [14]:
x.get_table()

,Name,Remaining events,Percentage of total events remaining,Relative percentage events,Remaining PFOs,Percentage of total PFOs remaining,Relative percentage of PFOs,Average PFOs per event
0,Initial data,429,100.0,100.0,1000,100.0,100.000000,2.331002
1,>2,429,100.0,100.0,685,68.5,68.500000,1.596737
2,<7,429,100.0,100.0,394,39.4,57.518248,0.918415
3,>=4,429,100.0,100.0,301,30.1,76.395939,0.701632


In [56]:
cutter2 = CutHandler()
small_data = data[evt_1][evt_2]
print(small_data)
smask_1 = small_data > 2
print(smask_1)
smask_2 = small_data < 7
print(smask_2)
smask_3 = small_data >=4
print(smask_3)
evts_cutter.add_mask(evt_1, "No empty")
print(evts_cutter.get_curr_mask())
print("1 done")
evts_cutter.add_mask(evt_2, " At most 3 in event")
print(evts_cutter.get_curr_mask())
print("2 done")
print(data[evts_cutter.get_curr_mask()])
print(evts_cutter._get_mask_signature(data))
print(evts_cutter._get_mask_signature(evts_cutter.apply_masks(data)))
print(evts_cutter.apply_masks(data))
print(data[evts_cutter.get_curr_mask()])
print(data)
print(evts_cutter.get_table())
evts_cutter.get_table(events_only=True)

(317, 688)

In [48]:
print(type(comb_cuts))
comb_cuts.get_table()

<class 'NotImplementedError'>


AttributeError: 'NotImplementedError' object has no attribute 'get_table'

In [69]:
print(cutter_num._get_mask_signature(mask_1))
print(cutter_num._get_mask_signature(mask_1, end=True))
print(cutter_num._get_mask_signature(mask_2))
print(cutter_num._get_mask_signature(mask_2, end=True))
print(cutter_num._get_mask_signature(mask_3))
print(cutter_num._get_mask_signature(mask_3, end=True))

(16, 30)
(11, 19)
(16, 30)
(13, 21)
(16, 30)
(9, 14)


In [25]:
(ak.sum(ak.any(mask_1, axis=1)), ak.sum(mask_1))

(25, 25)

In [29]:
ak.any(mask_1, axis=1)

<Array [[True], [True, ... [False, True]] type='16 * var * bool'>

In [30]:
mask_1

<Array [[[True]], ... [[False, True]]] type='16 * var * var * bool'>